### **1\. 콜백 기반 비동기 처리와 콜백 지옥(Callback Hell)**

**🔷 기본적인 콜백 비동기 패턴**

In [1]:
function fetchData(callback) {
  setTimeout(() => {
    callback(null, "데이터 로드 완료");
  }, 1000);
}

fetchData((err, data) => {
  if (err) {
    console.error("오류 발생:", err);
    return;
  }
  console.log(data);
});

데이터 로드 완료


🔷 콜백 지옥(Callback Hell)의 실제 문제 예시  
콜백 헬(Callback Hell)은 자바스크립트에서 비동기 작업을 콜백(callback)으로만 처리할 때 콜백 함수가 여러 번 중첩되면서 코드가 지나치게 복잡하고 읽기 어려워지는 현상을 말합니다.

In [2]:
setTimeout(() => {
  console.log("1초 후");
  setTimeout(() => {
    console.log("2초 후");
    setTimeout(() => {
      console.log("3초 후");
    }, 1000);
  }, 1000);
}, 1000);

2

1초 후
2초 후
3초 후


### **2\. Promise의 기본 구조와 체이닝**  
**🔷 Promise의 기본 구조**  
Promise는 생성과 동시에 비동기 작업을 시작하며, 작업이 완료되면 resolve 또는 reject가 호출되어 상태가 결정됩니다.  

**✔️ 예제 : 기본 Promise 생성 및 처리**

In [ ]:
// 비동기 작업을 Promise로 감싸서 반환하는 함수
function fetchData() {
  return new Promise((resolve, reject) => {
    // setTimeout으로 비동기 작업을 흉내 냄 (1초 후 실행)
    setTimeout(() => {
      // 정상적으로 작업을 마쳤을 때 resolve 호출
      resolve("데이터 로드 완료");

      // 오류 발생 시 reject를 호출하여 Promise를 실패 상태로 전환
      // reject(new Error("데이터 로드 실패"));
    }, 1000);
  });
}

// fetchData()는 Promise를 반환하므로 then/catch/finally를 연결할 수 있음
fetchData()
  .then(result => {
    // resolve가 호출된 경우 이 블록이 실행됨 (fulfilled 상태)
    console.log("성공:", result);

    //throw new Error("테스트 에러");
    // then() 내부에서 값을 반환하면 다음 then()으로 전달됨
    return "다음 처리 단계";
  })
  .catch(error => {
    // Promise가 reject되거나 then 내부에서 오류가 발생한 경우 실행됨
    console.error("오류:", error.message);
  })
  .finally(() => {
    // 성공/실패에 관계없이 항상 실행되는 마무리 블록
    console.log("작업 완료 (성공/실패 무관)");
  });

Promise { <pending> }

성공: 데이터 로드 완료
작업 완료 (성공/실패 무관)


**🔷 Promise 체이닝(Chaining)의 개념**

Promise는 then()을 호출할 때마다 새로운 Promise를 반환합니다.  
이 특성 덕분에 여러 비동기 작업을 자연스럽게 연결할 수 있습니다.

**✔️ 예제: 단계적 Promise 체이닝**



In [2]:
// 비동기 작업 1: 0.5초 후 완료되는 Promise 반환
function step1() {
  return new Promise(resolve => {
    setTimeout(() => {
      console.log("1단계 완료");
      resolve("step1-result"); // 다음 단계로 전달할 값
    }, 500);
  });
}

// 비동기 작업 2: 이전 단계의 결과(prev)를 받아 처리
function step2(prev) {
  return new Promise(resolve => {
    setTimeout(() => {
      console.log("2단계 완료, 이전 결과:", prev);
      resolve("step2-result"); // 다음 단계로 전달할 값
    }, 500);
  });
}

// 비동기 작업 3: step2의 결과(prev)를 전달받아 처리
function step3(prev) {
  return new Promise(resolve => {
    setTimeout(() => {
      console.log("3단계 완료, 이전 결과:", prev);
      resolve("step3-result"); // 전체 흐름의 최종 결과
    }, 500);
  });
}

// Promise 체이닝을 통해 순차적으로 비동기 작업 실행
step1()
  .then(result1 => step2(result1)) // step1의 결과를 step2로 전달
  .then(result2 => step3(result2)) // step2의 결과를 step3으로 전달
  .then(finalResult => {
    // 모든 단계가 정상적으로 완료된 경우
    console.log("최종 결과:", finalResult);
  })
  .catch(err => {
    // 어느 단계에서든 오류가 발생하면 이 블록으로 이동
    console.error("오류 발생:", err);
  });

Promise { <pending> }

1단계 완료
2단계 완료, 이전 결과: step1-result
3단계 완료, 이전 결과: step2-result
최종 결과: step3-result


**🔷 예외 처리(에러 전파)의 일관성**  
Promise의 가장 강력한 기능 중 하나는 에러 전파(error propagation)입니다.  
Promise 체이닝에서 어느 단계에서든 오류가 발생하면 그 이후의 then()은 건너뛰고, 가장 가까운 catch()로 이동합니다.

In [ ]:
Promise.resolve()
  .then(() => {
    throw new Error("중간 단계에서 오류 발생");
  })
  .then(() => {
    console.log("이 부분은 실행되지 않음");
  })
  .catch(err => {
    console.error("오류 처리:", err.message);
  });

오류 처리: 중간 단계에서 오류 발생


Promise { undefined }

### **3\. async/await과 현대 비동기 처리 실무 패턴** 
Promise를 통해 비동기 흐름을 개선할 수 있었지만, 여전히 then() 체이닝이 길어지거나 여러 비동기 작업을 조합하는 과정에서  
코드의 가독성과 의도 전달이 쉽지 않을 때가 있습니다.  
이러한 문제를 해소하기 위해 ES2017에서는 async/await 문법이 도입되었습니다.  
async/await은 Promise 기반으로 동작하면서도, 비동기 코드를 마치 동기 코드처럼 이해하고 작성할 수 있도록 설계된 문법입니다.  

**🔷 async/await - 가장 자연스러운 비동기 코드 작성 방식**  
async 키워드를 함수 앞에 붙이면 해당 함수는 자동으로 Promise를 반환하며,  
await 키워드는 Promise가 resolve될 때까지 함수 실행을 잠시 중단합니다.  
이 덕분에 비동기 흐름을 동기 코드처럼 읽고 작성할 수 있습니다.


In [ ]:
const delay = ms => new Promise(resolve => setTimeout(resolve, ms));

async function process() {
  console.log("시작");
  await delay(1000);
  console.log("1초 경과");
  await delay(1000);
  console.log("2초 경과");
}

console.log("--- process 함수 실행 시작 ---");
process();
console.log("--- process 함수 호출 완료 ---");
// 참고: process()가 Promise를 반환했으므로, 이 줄은 즉시 실행됩니다.

--- process 함수 실행 시작 ---
시작
--- process 함수 호출 완료 ---


1초 경과
2초 경과


**🔷 비동기 에러 처리 패턴 - try/catch와 async 조합**  
Promise 기반 코드의 오류는 catch()로 처리해야 했지만 async/await은 기존 구조와 동일하게 try/catch로 에러 처리가 가능합니다.


In [3]:
async function fetchUser() {
  throw new Error("사용자 정보를 가져오는 중 오류 발생");
}

async function main() {
  try {
    const user = await fetchUser();
    console.log(user);
  } catch (err) {
    console.error("에러 발생:", err.message);
  }
}

main();

에러 발생: 사용자 정보를 가져오는 중 오류 발생


Promise { undefined }

**🔷 병렬 실행을 위한 Promise 제어 메서드 - Promise.all / allSettled / race / any**  

**1\. Promise.all - 모두 성공해야 완료 (Fail-Fast)**  


In [ ]:
async function executeAll() {
  console.log("--- 1. Promise.all 실행 ---");
  try {
    const results = await Promise.all([
      // 1초 후 성공
      new Promise(resolve => setTimeout(() => resolve("A 결과"), 1000)),
      // 0.5초 후 성공
      new Promise(resolve => setTimeout(() => resolve("B 결과"), 500)),
      // 2초 후 성공
      new Promise(resolve => setTimeout(() => resolve("C 결과"), 2000))
    ]);

    console.log("✅ Promise.all 성공 (모두 완료):", results);
  } catch (error) {
    console.log("❌ Promise.all 실패 (하나라도 실패 시):", error);
  }

  // 실패 예시 (가장 먼저 실패하는 'Reject'가 반환됨)
  try {
    await Promise.all([
      new Promise((_, reject) => setTimeout(() => reject("첫 번째 실패"), 100)),
      new Promise(resolve => setTimeout(() => resolve("두 번째 성공"), 2000))
    ]);
  } catch (error) {
    console.log("❌ Promise.all 실패 (단 하나 실패):", error); // "첫 번째 실패" 출력
  }
}

 executeAll();

--- 1. Promise.all 실행 ---


Promise { <pending> }

✅ Promise.all 성공 (모두 완료): [ "A 결과", "B 결과", "C 결과" ]
❌ Promise.all 실패 (단 하나 실패): 첫 번째 실패


**2\. Promise.allSettled - 성공/실패 상관없이 모두 결과 반환**

In [ ]:
async function executeAllSettled() {
  console.log("\n--- 2. Promise.allSettled 실행 ---");
  const result = await Promise.allSettled([
    Promise.resolve("성공 결과"),
    Promise.reject(new Error("네트워크 오류")),
    new Promise(resolve => setTimeout(() => resolve(123), 100))
  ]);

  console.log("✅ Promise.allSettled 결과 (성공/실패 모두 포함):");
  // 결과는 항상 { status, value/reason } 객체 배열입니다.
  console.log(result);
}

executeAllSettled();

**3\. Promise.race - 가장 먼저 끝난 Promise의 결과 사용** 

In [ ]:
async function executeRace() {
  console.log("\n--- 3. Promise.race 실행 ---");

  const fastSuccess = new Promise(resolve => setTimeout(() => resolve("1. 🚀 가장 빠른 성공"), 100));
  const slowReject = new Promise((_, reject) => setTimeout(() => reject("2. 🐢 느린 실패"), 500));
  const slowSuccess = new Promise(resolve => setTimeout(() => resolve("3. 🐌 느린 성공"), 2000));

  try {
    const result = await Promise.race([fastSuccess, slowReject, slowSuccess]);
    console.log("✅ Promise.race 결과:", result); // "1. 🚀 가장 빠른 성공"이 출력됨
  } catch (error) {
    console.log("❌ Promise.race 실패:", error); // 가장 먼저 끝난 게 실패면 실패가 반환됨
  }
}

executeRace();

**4\. Promise.any - 가장 먼저 성공한 Promise 반환**

In [ ]:
async function executeAny() {
  console.log("\n--- 4. Promise.any 실행 ---");

  // 성공 예시: 'OK'가 가장 먼저 성공합니다.
  const successResult = await Promise.any([
    Promise.reject("X"), // 1. 실패
    Promise.resolve("OK"), // 2. 성공 (가장 먼저 성공)
    Promise.reject("Y") // 3. 실패
  ]);
  console.log("✅ Promise.any 성공 (가장 빠른 성공):", successResult); // "OK"

  // 모두 실패 예시: AggregateError 발생
  const reject1 = new Promise((_, reject) => setTimeout(() => reject("첫 번째 실패"), 200));
  // 가장 먼저 종료되지만, 실패이므로 무시
  const reject2 = new Promise((_, reject) => setTimeout(() => reject("두 번째 실패"), 100));

  try {
    await Promise.any([reject1, reject2]);
  } catch (error) {
    console.log("❌ Promise.any 실패 (모두 실패 시):");
    // AggregateError는 Array.prototype.errors에 실패 이유들을 담습니다.
    console.log("오류 타입:", error.name);
    console.log("모든 실패 이유:", error.errors);
  }
}

executeAny();

: 

### **4\. AbortController / AbortSignal (ES2022)**  
AbortController와 AbortSignal은 진행 중인 비동기 작업(Asynchronous Operations)을 중간에 취소(Cancellation)하기 위한 공식 표준 API입니다.  
이전에는 비동기 작업을 취소하는 표준화된 방법이 없었으나, 이 API가 도입되면서 일관된 취소 메커니즘을 제공하게 되었습니다. (ES2022 표준)  

**✔️ 예제: fetch 요청 취소하기 (Node.js 18+)**

In [ ]:
const controller = new AbortController();
const fetchURL = "https://httpbin.org/delay/2"; // 응답까지 2초가 걸리는 URL

// 0.5초(500ms) 후에 취소 명령을 내립니다.
const timeoutId = setTimeout(() => {
  controller.abort();
  console.log("🚩 요청 취소 명령어 발동 (Timeout: 500ms)");
}, 500);

async function runFetchWithCancellation() {
  console.log(`⏳ ${fetchURL} 에 요청 시작...`);
  try {
    const response = await fetch(fetchURL, {
      // AbortSignal을 fetch 요청에 연결합니다.
      signal: controller.signal
    });

    // 요청이 취소되지 않고 성공했을 때만 timeout을 클리어합니다.
    clearTimeout(timeoutId);

    console.log("✅ 응답 성공!");
    const text = await response.text();
    //console.log(text); // 긴 응답 내용이므로 주석 처리
  } catch (err) {
    // 취소되면 catch 블록으로 들어오며 'AbortError'가 발생합니다.
    if (err.name === 'AbortError') {
      console.error("❌ 에러:", err.name, "- 요청이 취소되었습니다.");
    } else {
      console.error("❌ 에러:", err.name, err.message);
    }
  }
}

// runFetchWithCancellation();

**✔️ setTimeout 및 Promise와 결합하여 취소 가능한 작업 만들기**

In [4]:
// 주어진 Signal이 abort될 때까지 기다리거나, 지정된 시간(ms)만큼 지연하는 함수
function delayWithSignal(ms, signal) {
  return new Promise((resolve, reject) => {
    // 1. Abort 되었을 때의 핸들러 등록
    const onAbort = () => {
      clearTimeout(timerId); // 지연 타이머를 클리어
      reject(new DOMException("Delay aborted", "AbortError"));
    };

    // Signal이 이미 취소된 상태라면 즉시 reject
    if (signal.aborted) {
      onAbort();
      return;
    }

    // 2. 'abort' 이벤트 리스너 등록
    signal.addEventListener('abort', onAbort, { once: true });

    // 3. 지연 타이머 설정
    const timerId = setTimeout(() => {
      signal.removeEventListener('abort', onAbort); // 성공 시 리스너 제거
      resolve(`✅ ${ms}ms 지연 완료`);
    }, ms);
  });
}

async function testDelayCancellation() {
    console.log("\n--- 취소 가능한 지연 테스트 ---");
    const controller = new AbortController();

    // 100ms 후 취소 명령
    setTimeout(() => controller.abort(), 100);

    try {
        // 500ms 지연 요청
        const result = await delayWithSignal(500, controller.signal);
        console.log(result);
    } catch (err) {
        console.error("❌ 취소 에러:", err.name, "-", err.message); // AbortError
    }
}

testDelayCancellation();


--- 취소 가능한 지연 테스트 ---


Promise { <pending> }

❌ 취소 에러: AbortError - Delay aborted


### **5\. Microtask / Task Queue와 Event Loop**  

**✔️ Event Loop 우선순위 규칙 (실행 순서)**  
이벤트 루프는 무한히 반복하면서 다음의 명확한 규칙에 따라 작업을 수행합니다.  
1\. 동기 코드 실행: Call Stack이 완전히 비워질 때까지 모든 코드를 실행합니다.  
2\. Microtask 처리: Call Stack이 비워지면, Microtask Queue에 있는 작업을 모두 비워질 때까지 연속적으로 실행합니다.  
3\. Task 하나 실행: Microtask Queue가 비워지면, Task Queue에서 작업 딱 1개를 꺼내 Call Stack으로 옮겨 실행합니다.  
4\. Microtask 재처리: Task 1개가 완료되면, 다시 Microtask Queue를 비우는 작업을 수행합니다.  
5\. 반복: 3번과 4번을 반복합니다.  
이 규칙 때문에 Promise 기반 비동기 작업(Microtask)은 일반적인 타이머 기반 작업(Task)보다 훨씬 더 빠른 실행 시점을 갖게 됩니다.  

**✔️ 예제: Microtask 우선순위 확인**


In [5]:
console.log("시작"); // 1. 동기 코드

setTimeout(() => {
  console.log("setTimeout 실행 (Task)"); // 4. Task Queue에 등록됨
}, 0);

Promise.resolve().then(() => {
  console.log("Promise then 실행 (Microtask)"); // 3. Microtask Queue에 등록됨
});

console.log("끝"); // 2. 동기 코드

시작
끝
Promise then 실행 (Microtask)
setTimeout 실행 (Task)


1\. 시작과 끝이 동기 코드로서 Call Stack에서 즉시 실행됩니다. (Call Stack Clear)    
2\. Call Stack이 비워지면, 이벤트 루프가 Microtask Queue를 확인합니다.   
3\. Promise.resolve().then(...) 콜백이 Microtask Queue에 있으므로 이것이 실행됩니다.  
4\. Microtask Queue가 비워지면, 이벤트 루프가 Task Queue를 확인합니다.  
5\. setTimeout(...) 콜백이 Task Queue에 있으므로 이것이 실행됩니다.

### **6\. Promise.withResolvers(ES2024) - 외부 제어형 Promise의 공식 해법**  
Promise.withResolvers()는 2024년(ES2024)에 공식 도입된 새로운 Promise 생성 API입니다.  
이는 Deferred 패턴이라고 불렸던, Promise를 외부에서 성공(resolve) 또는 실패(reject) 상태로 만들 필요가 있을 때 사용하는 가장 현대적이고 안전한 방법입니다.

**✔️ Timeout 제어 Promise 구현**  
특정 시간이 지난 후 성공하는 Promise를 만들 때, resolve를 setTimeout 내부에 깔끔하게 전달할 수 있습니다.



In [6]:
function timeout(ms) {
  // Promise와 resolve 함수를 즉시 얻습니다.
  const { promise, resolve } = Promise.withResolvers();

  // 지정된 ms 후에 resolve를 호출하여 promise를 성공시킵니다.
  setTimeout(() => {
    resolve(`Timeout ${ms}ms 경과`);
  }, ms);

  return promise; // 외부에서 await 할 수 있는 promise를 반환
}

async function runTimeoutExample() {
  console.log("⏳ 1초 대기 시작");
  const msg = await timeout(1000); // 1초 대기
  console.log(msg); // Timeout 1000ms 경과
}

runTimeoutExample();

⏳ 1초 대기 시작


Promise { <pending> }

Timeout 1000ms 경과


**✔️ 이벤트 기반 Deferred 패턴 (이벤트 발생 대기)**  
특정 DOM 이벤트나 네트워크 이벤트가 발생할 때까지 await 상태로 대기하는 Promise를 안전하게 만들 수 있습니다

In [ ]:
// target 객체에서 eventName 이벤트가 발생할 때까지 대기
function waitForEvent(target, eventName) {
  const { promise, resolve } = Promise.withResolvers();

  function handler(e) {
    // 1. 이벤트가 발생하면 리스너를 제거하고
    target.removeEventListener(eventName, handler);
    // 2. resolve를 호출하여 promise를 성공시킵니다.
    resolve(e);
  }

  target.addEventListener(eventName, handler);

  return promise;
}

// 예시 (브라우저 환경)
// async function handleUserClick() {
//    console.log("다음 클릭을 기다립니다...");
//    const clickEvent = await waitForEvent(document, 'click');
//    console.log('클릭 감지!', clickEvent.target);
// }
// handleUserClick();

**✔️ 외부에서 제어 가능한 작업 상태 관리**  
여러 비동기 흐름이 하나의 상태를 공유하고 특정 시점에만 다음 단계로 넘어가야 할 때 유용합니다.

In [7]:
// 초기화 작업이 완료될 때까지 기다릴 수 있는 Promise를 미리 생성
const initState = Promise.withResolvers();

async function initialize() {
  console.log("⏳ 초기화 중... (1초 소요)");
  await new Promise(r => setTimeout(r, 1000));
  // 초기화가 완료되는 시점에 resolve를 호출합니다.
  initState.resolve("✨ 초기화 완료 상태!");
}

async function requestHandler() {
  console.log("⚙️ 요청 대기 중...");
  // initState.promise가 resolve될 때까지 모든 요청은 대기합니다.
  const status = await initState.promise;
  console.log(`✅ 요청 처리 시작: ${status}`);
}

initialize(); // 초기화 작업 시작
requestHandler(); // 요청 처리 함수 호출 (초기화 완료될 때까지 대기)

⏳ 초기화 중... (1초 소요)
⚙️ 요청 대기 중...


Promise { <pending> }

✅ 요청 처리 시작: ✨ 초기화 완료 상태!
